# 🥔 Phase 5: Model Selection & Architecture Design

This notebook defines the deep learning model architecture for potato disease classification.

**Features:**
- Transfer learning with pre-trained models (MobileNetV2, EfficientNet, ResNet)
- Custom classification head for 3-class potato disease detection
- Mobile-optimized architecture options
- Regularization techniques to prevent overfitting
- Model comparison and selection

## 1. Mount Google Drive & Setup

In [ ]:
from google.colab import drive
import os
import shutil

MOUNT_PATH = '/content/drive'

def mount_drive():
    if os.path.exists(os.path.join(MOUNT_PATH, 'MyDrive')):
        print('✅ Google Drive is already mounted!')
        return True
    if os.path.exists(MOUNT_PATH):
        try:
            drive.flush_and_unmount()
        except:
            pass
        if os.path.exists(MOUNT_PATH):
            try:
                shutil.rmtree(MOUNT_PATH)
            except:
                pass
    try:
        drive.mount(MOUNT_PATH)
        print('✅ Google Drive mounted successfully!')
        return True
    except Exception as e:
        print(f'❌ Mount failed: {e}')
        return False

mount_drive()

In [ ]:
# Install and import dependencies
!pip install -q tensorflow keras matplotlib seaborn

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from datetime import datetime

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.applications import (
    MobileNetV2,
    EfficientNetB0,
    ResNet50,
    VGG16
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

print(f"✅ TensorFlow version: {tf.__version__}")
print(f"✅ Keras version: {keras.__version__}")

# Check GPU
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"🚀 GPU detected: {gpus[0].name}")
else:
    print("⚠️ No GPU detected, using CPU")

## 2. Configuration

In [ ]:
# ===== CONFIGURATION =====

# Dataset paths
DATASET_DIR = "/content/drive/MyDrive/DrukFarm/data/final_potato_dataset"
MODEL_SAVE_DIR = "/content/drive/MyDrive/DrukFarm/models"

# Model parameters
IMG_SIZE = (224, 224)       # Input image size
IMG_SHAPE = (224, 224, 3)   # Including channels
NUM_CLASSES = 3             # Early_Blight, Late_Blight, Healthy
CLASS_NAMES = ['Early_Blight', 'Healthy', 'Late_Blight']  # Alphabetical order

# Training parameters (for reference)
BATCH_SIZE = 32
LEARNING_RATE = 0.0001
EPOCHS = 50

# Create model save directory
os.makedirs(MODEL_SAVE_DIR, exist_ok=True)

print("✅ Configuration loaded!")
print(f"\n📁 Dataset: {DATASET_DIR}")
print(f"📂 Models will be saved to: {MODEL_SAVE_DIR}")
print(f"\n📊 Model parameters:")
print(f"   • Input size: {IMG_SIZE}")
print(f"   • Number of classes: {NUM_CLASSES}")
print(f"   • Classes: {CLASS_NAMES}")

## 3. Model Architecture: MobileNetV2 (Recommended)

**Why MobileNetV2?**
- ✅ Lightweight & fast inference
- ✅ Optimized for mobile/edge deployment
- ✅ Excellent accuracy-to-size ratio
- ✅ Pre-trained on ImageNet (1.4M images)

In [ ]:
def create_mobilenet_model(input_shape, num_classes, fine_tune_layers=20):
    """
    Create MobileNetV2-based model for potato disease classification.
    
    Architecture:
    - Base: MobileNetV2 (pre-trained on ImageNet)
    - Head: Global Average Pooling → Dense(256) → Dropout → Dense(num_classes)
    """
    # Load pre-trained MobileNetV2 (without top classification layers)
    base_model = MobileNetV2(
        input_shape=input_shape,
        include_top=False,
        weights='imagenet'
    )
    
    # Freeze base model layers (except last fine_tune_layers)
    base_model.trainable = True
    for layer in base_model.layers[:-fine_tune_layers]:
        layer.trainable = False
    
    # Build classification head
    model = models.Sequential([
        # Input preprocessing
        layers.InputLayer(input_shape=input_shape),
        
        # Preprocessing (MobileNetV2 specific)
        layers.Rescaling(1./127.5, offset=-1),  # Scale to [-1, 1]
        
        # Feature extraction backbone
        base_model,
        
        # Classification head
        layers.GlobalAveragePooling2D(),
        layers.BatchNormalization(),
        layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
        layers.Dropout(0.5),
        layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
        layers.Dropout(0.3),
        layers.Dense(num_classes, activation='softmax', name='output')
    ], name='PotatoDisease_MobileNetV2')
    
    return model, base_model


# Create model
mobilenet_model, mobilenet_base = create_mobilenet_model(IMG_SHAPE, NUM_CLASSES)

# Display summary
print("\n" + "=" * 60)
print("📱 MobileNetV2 Architecture")
print("=" * 60)
mobilenet_model.summary()

# Count parameters
trainable_params = int(sum([tf.keras.backend.count_params(w) for w in mobilenet_model.trainable_weights]))
non_trainable_params = int(sum([tf.keras.backend.count_params(w) for w in mobilenet_model.non_trainable_weights]))
print(f"\n📊 Parameters:")
print(f"   • Trainable: {trainable_params:,}")
print(f"   • Non-trainable: {non_trainable_params:,}")
print(f"   • Total: {trainable_params + non_trainable_params:,}")

## 4. Alternative: EfficientNetB0 (Higher Accuracy)

**Why EfficientNetB0?**
- ✅ State-of-the-art accuracy
- ✅ Compound scaling architecture
- ✅ More efficient than ResNet/VGG
- ⚠️ Slightly larger than MobileNetV2

In [ ]:
def create_efficientnet_model(input_shape, num_classes, fine_tune_layers=30):
    """
    Create EfficientNetB0-based model for potato disease classification.
    """
    # Load pre-trained EfficientNetB0
    base_model = EfficientNetB0(
        input_shape=input_shape,
        include_top=False,
        weights='imagenet'
    )
    
    # Freeze base model layers
    base_model.trainable = True
    for layer in base_model.layers[:-fine_tune_layers]:
        layer.trainable = False
    
    # Build model
    model = models.Sequential([
        layers.InputLayer(input_shape=input_shape),
        
        # EfficientNet has built-in preprocessing
        base_model,
        
        # Classification head
        layers.GlobalAveragePooling2D(),
        layers.BatchNormalization(),
        layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax', name='output')
    ], name='PotatoDisease_EfficientNetB0')
    
    return model, base_model


# Create model
efficientnet_model, efficientnet_base = create_efficientnet_model(IMG_SHAPE, NUM_CLASSES)

print("\n" + "=" * 60)
print("⚡ EfficientNetB0 Architecture")
print("=" * 60)

# Count parameters
trainable_params = int(sum([tf.keras.backend.count_params(w) for w in efficientnet_model.trainable_weights]))
non_trainable_params = int(sum([tf.keras.backend.count_params(w) for w in efficientnet_model.non_trainable_weights]))
print(f"\n📊 Parameters:")
print(f"   • Trainable: {trainable_params:,}")
print(f"   • Non-trainable: {non_trainable_params:,}")
print(f"   • Total: {trainable_params + non_trainable_params:,}")

## 5. Alternative: Custom Lightweight CNN

**Why Custom CNN?**
- ✅ Smallest model size
- ✅ Fastest inference
- ✅ Full control over architecture
- ⚠️ May need more training data

In [ ]:
def create_custom_cnn(input_shape, num_classes):
    """
    Create a custom lightweight CNN from scratch.
    Optimized for small datasets and fast inference.
    """
    model = models.Sequential([
        layers.InputLayer(input_shape=input_shape),
        
        # Normalization
        layers.Rescaling(1./255),
        
        # Block 1: 32 filters
        layers.Conv2D(32, (3, 3), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.Conv2D(32, (3, 3), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Block 2: 64 filters
        layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Block 3: 128 filters
        layers.Conv2D(128, (3, 3), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.Conv2D(128, (3, 3), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Block 4: 256 filters
        layers.Conv2D(256, (3, 3), padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Classification head
        layers.GlobalAveragePooling2D(),
        layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
        layers.Dropout(0.5),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(num_classes, activation='softmax', name='output')
    ], name='PotatoDisease_CustomCNN')
    
    return model


# Create model
custom_model = create_custom_cnn(IMG_SHAPE, NUM_CLASSES)

print("\n" + "=" * 60)
print("🔧 Custom Lightweight CNN Architecture")
print("=" * 60)
custom_model.summary()

# Count parameters
total_params = int(custom_model.count_params())
print(f"\n📊 Total Parameters: {total_params:,}")

## 6. Model Comparison

In [ ]:
def compare_models(models_dict):
    """
    Compare different model architectures.
    """
    comparison = []
    
    for name, model in models_dict.items():
        total_params = int(model.count_params())
        trainable = int(sum([tf.keras.backend.count_params(w) for w in model.trainable_weights]))
        
        # Estimate model size (4 bytes per parameter)
        size_mb = (total_params * 4) / (1024 * 1024)
        
        comparison.append({
            'Model': name,
            'Total Params': f"{total_params:,}",
            'Trainable Params': f"{trainable:,}",
            'Size (MB)': f"{size_mb:.1f}",
            'Layers': len(model.layers)
        })
    
    return comparison


# Compare all models
models_to_compare = {
    'MobileNetV2': mobilenet_model,
    'EfficientNetB0': efficientnet_model,
    'Custom CNN': custom_model
}

comparison = compare_models(models_to_compare)

print("\n" + "=" * 70)
print("📊 MODEL COMPARISON")
print("=" * 70)

# Print as table
print(f"\n{'Model':<18} {'Total Params':<15} {'Trainable':<15} {'Size (MB)':<12} {'Layers'}")
print("-" * 70)
for c in comparison:
    print(f"{c['Model']:<18} {c['Total Params']:<15} {c['Trainable Params']:<15} {c['Size (MB)']:<12} {c['Layers']}")

print("\n📋 RECOMMENDATION:")
print("   🏆 MobileNetV2 - Best balance of accuracy and efficiency")
print("   ⚡ EfficientNetB0 - Highest accuracy potential")
print("   🔧 Custom CNN - Smallest size, fastest inference")

## 7. Visualize Model Architecture

In [ ]:
from tensorflow.keras.utils import plot_model

# Plot MobileNetV2 architecture
print("\n📐 MobileNetV2 Architecture Diagram")
try:
    plot_model(
        mobilenet_model, 
        to_file='/content/mobilenet_architecture.png',
        show_shapes=True,
        show_layer_names=True,
        dpi=100
    )
    from IPython.display import Image, display
    display(Image('/content/mobilenet_architecture.png'))
except Exception as e:
    print(f"Could not generate diagram: {e}")
    print("(This requires graphviz to be installed)")

## 8. Select Final Model & Compile

In [ ]:
# ===== SELECT YOUR MODEL HERE =====
# Options: 'mobilenet', 'efficientnet', 'custom'

SELECTED_MODEL = 'mobilenet'  # Change this to select different model

# Get the selected model
if SELECTED_MODEL == 'mobilenet':
    final_model = mobilenet_model
    model_name = 'PotatoDisease_MobileNetV2'
elif SELECTED_MODEL == 'efficientnet':
    final_model = efficientnet_model
    model_name = 'PotatoDisease_EfficientNetB0'
else:
    final_model = custom_model
    model_name = 'PotatoDisease_CustomCNN'

print(f"\n✅ Selected Model: {model_name}")

# Compile the model
final_model.compile(
    optimizer=Adam(learning_rate=LEARNING_RATE),
    loss='categorical_crossentropy',
    metrics=['accuracy', 
             tf.keras.metrics.Precision(name='precision'),
             tf.keras.metrics.Recall(name='recall')]
)

print("\n📋 Model compiled with:")
print(f"   • Optimizer: Adam (lr={LEARNING_RATE})")
print(f"   • Loss: Categorical Crossentropy")
print(f"   • Metrics: Accuracy, Precision, Recall")

## 9. Define Training Callbacks

In [ ]:
def create_callbacks(model_save_path):
    """
    Create training callbacks for:
    - Early stopping to prevent overfitting
    - Model checkpointing to save best weights
    - Learning rate reduction on plateau
    """
    callbacks = [
        # Early stopping
        EarlyStopping(
            monitor='val_loss',
            patience=10,
            restore_best_weights=True,
            verbose=1
        ),
        
        # Save best model
        ModelCheckpoint(
            filepath=model_save_path,
            monitor='val_accuracy',
            save_best_only=True,
            save_weights_only=False,
            verbose=1
        ),
        
        # Reduce LR on plateau
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.2,
            patience=5,
            min_lr=1e-7,
            verbose=1
        )
    ]
    
    return callbacks


# Create callbacks
model_save_path = os.path.join(MODEL_SAVE_DIR, f"{model_name}_best.keras")
callbacks = create_callbacks(model_save_path)

print("\n✅ Training callbacks configured:")
print(f"   • EarlyStopping (patience=10)")
print(f"   • ModelCheckpoint → {model_save_path}")
print(f"   • ReduceLROnPlateau (factor=0.2)")

## 10. Save Architecture Configuration

In [ ]:
def save_architecture_config(model, model_name, save_dir):
    """
    Save model architecture configuration for documentation.
    """
    # Convert numpy int64 to Python int for JSON serialization
    total_params = int(model.count_params())
    trainable_params = int(sum([tf.keras.backend.count_params(w) for w in model.trainable_weights]))
    num_layers = int(len(model.layers))
    
    config = {
        'phase': 'Phase 5: Model Selection & Architecture Design',
        'timestamp': datetime.now().isoformat(),
        'model_name': model_name,
        'architecture': {
            'input_shape': list(IMG_SHAPE),
            'num_classes': int(NUM_CLASSES),
            'class_names': CLASS_NAMES,
            'total_params': total_params,
            'trainable_params': trainable_params,
            'layers': num_layers
        },
        'training_config': {
            'optimizer': 'Adam',
            'learning_rate': float(LEARNING_RATE),
            'loss': 'categorical_crossentropy',
            'batch_size': int(BATCH_SIZE),
            'epochs': int(EPOCHS)
        },
        'callbacks': [
            'EarlyStopping (patience=10)',
            'ModelCheckpoint (best val_accuracy)',
            'ReduceLROnPlateau (factor=0.2)'
        ],
        'regularization': [
            'L2 regularization (0.01)',
            'Dropout (0.3-0.5)',
            'Batch Normalization'
        ]
    }
    
    # Save config
    config_path = os.path.join(save_dir, 'architecture_config.json')
    with open(config_path, 'w') as f:
        json.dump(config, f, indent=2)
    
    # Save model architecture (JSON)
    arch_path = os.path.join(save_dir, f'{model_name}_architecture.json')
    with open(arch_path, 'w') as f:
        f.write(model.to_json())
    
    return config


# Save configuration
arch_config = save_architecture_config(final_model, model_name, MODEL_SAVE_DIR)

print("\n" + "=" * 60)
print("📋 ARCHITECTURE CONFIGURATION SAVED")
print("=" * 60)
print(f"\n✅ Config saved to: {MODEL_SAVE_DIR}/architecture_config.json")
print(f"✅ Architecture saved to: {MODEL_SAVE_DIR}/{model_name}_architecture.json")

## 11. Architecture Summary

In [ ]:
# Get parameters as Python int for display
total_p = int(final_model.count_params())
trainable_p = int(sum([tf.keras.backend.count_params(w) for w in final_model.trainable_weights]))

print("\n" + "=" * 70)
print("📋 PHASE 5 SUMMARY: Model Architecture Design")
print("=" * 70)

print(f"""
🏆 SELECTED MODEL: {model_name}

📐 ARCHITECTURE:
   ┌─────────────────────────────────────────────────────────────┐
   │  Input Layer (224 × 224 × 3)                                │
   ├─────────────────────────────────────────────────────────────┤
   │  Preprocessing (Rescaling)                                  │
   ├─────────────────────────────────────────────────────────────┤
   │  MobileNetV2 Base (ImageNet pretrained)                     │
   │  - Last 20 layers fine-tuned                                │
   ├─────────────────────────────────────────────────────────────┤
   │  Global Average Pooling 2D                                  │
   ├─────────────────────────────────────────────────────────────┤
   │  BatchNormalization                                         │
   ├─────────────────────────────────────────────────────────────┤
   │  Dense(256, ReLU) + L2 Regularization                       │
   ├─────────────────────────────────────────────────────────────┤
   │  Dropout(0.5)                                               │
   ├─────────────────────────────────────────────────────────────┤
   │  Dense(128, ReLU) + L2 Regularization                       │
   ├─────────────────────────────────────────────────────────────┤
   │  Dropout(0.3)                                               │
   ├─────────────────────────────────────────────────────────────┤
   │  Dense(3, Softmax) → Output                                 │
   └─────────────────────────────────────────────────────────────┘

📊 PARAMETERS:
   • Total: {total_p:,}
   • Trainable: {trainable_p:,}

🛡️ REGULARIZATION:
   • L2 weight decay (λ=0.01)
   • Dropout (0.3-0.5)
   • Batch Normalization
   • Early Stopping

📂 OUTPUT CLASSES:
   0 → Early_Blight
   1 → Healthy  
   2 → Late_Blight

✅ Model is ready for training in Phase 6!
""")

---

## ✅ Phase 5 Complete!

**Selected Architecture:** MobileNetV2 with custom classification head

**Key Design Decisions:**
- ✅ Transfer learning from ImageNet
- ✅ Fine-tuning last 20 layers
- ✅ Global Average Pooling (reduces overfitting)
- ✅ Heavy dropout regularization
- ✅ L2 weight regularization
- ✅ Mobile-deployment ready

**Saved Files:**
```
/content/drive/MyDrive/DrukFarm/models/
├── architecture_config.json
└── PotatoDisease_MobileNetV2_architecture.json
```

**Next Steps:**
- Phase 6: Model Training & Evaluation